In [1]:
cd(@__DIR__) #makes the directory where this script is located the new working directory
using Pkg
Pkg.activate()
Pkg.instantiate()
using MixedModels
using RCall
using DataFrames
using CSV
using RData
using Statistics
using Dates
using HypothesisTests

 Activating environment at `C:\Users\bjoer\.julia\environments\v1.4\Project.toml`
┌ Info: Precompiling MixedModels [ff71e718-51f3-5ec2-a782-8ffcbfa3c316]
└ @ Base loading.jl:1260



R version 3.6.0 (2019-04-26) -- "Planting of a Tree"
Copyright (C) 2019 The R Foundation for Statistical Computing
Platform: x86_64-w64-mingw32/x64 (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.



┌ Warning: RCall.jl: This version of Shiny is designed to work with 'htmlwidgets' >= 1.5.
│     Please upgrade via install.packages('htmlwidgets').
│ Registered S3 method overwritten by 'xts':
│   method     from
│   as.zoo.xts zoo 
│ [Previously saved workspace restored]
│ 
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113


ArgumentError: ArgumentError: Package HypothesisTests not found in current path:
- Run `import Pkg; Pkg.add("HypothesisTests")` to install the HypothesisTests package.


In [2]:
R"""
require(dplyr, quietly = TRUE)   # for data wrangling
require(tidyverse, quietly = TRUE)   # for data wrangling
require(lme4)
require(lmerTest)
require(quickpsy)

SimulatePsychometricFunction_Staircase = function(ID, 
                                                  ConditionOfInterest, 
                                                  StandardValues, 
                                                  reps, 
                                                  PSE_Difference, 
                                                  JND_Difference, 
                                                  Multiplicator_PSE_Standard, 
                                                  Multiplicator_SD_Standard, 
                                                  SD_ResponseFunction, 
                                                  Mean_Variability_Between = 0.1, 
                                                  SD_Variability_Between = 0.1){
  Psychometric = expand.grid(ID=ID, ConditionOfInterest=ConditionOfInterest, StandardValues=StandardValues, reps = reps)
  
  Psychometric = Psychometric %>%
    group_by(ID) %>%#
    mutate(PSE_Factor_ID = rnorm(1,1,Mean_Variability_Between),
           SD_Factor_ID = rnorm(1,1,SD_Variability_Between))
  
  Psychometric = Psychometric %>%
    mutate(
      Mean_Standard = StandardValues+StandardValues*Multiplicator_PSE_Standard,
      SD_Standard = StandardValues*Multiplicator_SD_Standard,
      Mean = (Mean_Standard + (ConditionOfInterest==ConditionOfInterest[2])*StandardValues*PSE_Difference)*PSE_Factor_ID,
      SD = abs((SD_Standard + (ConditionOfInterest==ConditionOfInterest[2])*SD_Standard*JND_Difference)*SD_Factor_ID),
      staircase_factor = rcauchy(length(reps),1,SD_ResponseFunction), 
      Presented_TestStimulusStrength = Mean*staircase_factor,
      Difference = Presented_TestStimulusStrength - StandardValues,
      AnswerProbability = pnorm(Presented_TestStimulusStrength,Mean,SD),
      Answer = as.numeric(rbernoulli(length(AnswerProbability),AnswerProbability))
    )
  
  Psychometric = Psychometric %>%
    filter(abs(staircase_factor-1) < 0.75) %>%
    group_by(ID,ConditionOfInterest,StandardValues,Difference) %>%
    mutate(Yes = sum(Answer==1),
           Total = length(ConditionOfInterest))
  
  Psychometric
}
""";

┌ Warning: RCall.jl: Warning: package 'dplyr' was built under R version 3.6.3
│ 
│ Attaching package: 'dplyr'
│ 
│ The following objects are masked from 'package:stats':
│ 
│     filter, lag
│ 
│ The following objects are masked from 'package:base':
│ 
│     intersect, setdiff, setequal, union
│ 
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: -- Attaching packages --------------------------------------- tidyverse 1.2.1 --
│ v ggplot2 3.2.0     v readr   1.3.1
│ v tibble  3.0.0     v purrr   0.3.3
│ v tidyr   1.0.2     v stringr 1.4.0
│ v ggplot2 3.2.0     v forcats 0.4.0
│ Warning: package 'tibble' was built under R version 3.6.3
│ Warning: package 'tidyr' was built under R version 3.6.3
│ Warning: package 'purrr' was built under R version 3.6.3
│ -- Conflicts ------------------------------------------ tidyverse_conflicts() --
│ x dplyr::filter() masks stats::filter()
│ x dplyr::lag()    masks stats::lag()
└ @ RCall C:\Users\bjoer\.julia\package

In [2]:
function SimulateDataframe_Twolevel(n,
        ConditionOfInterest,
        StandardValues,
        reps,
        PSE_Difference,
        JND_Difference,
        Multiplicator_PSE_Standard,
        Multiplicator_SD_Standard,
        SD_ResponseFunction,
        Mean_Variability_Between,
        SD_Variability_Between)
    
    @rput n ConditionOfInterest StandardValues reps PSE_Difference JND_Difference Multiplicator_PSE_Standard Multiplicator_SD_Standard SD_ResponseFunction Mean_Variability_Between SD_Variability_Between

    R"""
    ID = paste0("s",1:n)
        Psychometric = SimulatePsychometricFunction_Staircase(ID,
            ConditionOfInterest,
            StandardValues,
            1:reps,
            PSE_Difference,
            JND_Difference,
            Multiplicator_PSE_Standard,
            Multiplicator_SD_Standard,
            SD_ResponseFunction,
            Mean_Variability_Between,
            SD_Variability_Between)
    
        Parameters = quickpsy(Psychometric,Difference,Answer,grouping = .(ID,ConditionOfInterest,StandardValues), bootstrap = "none")$par
        Parameters2 = Parameters %>%
        filter(parn == "p1") %>%
        select(ID,ConditionOfInterest,Mean=par, StandardValues)
        Parameters2$SD = Parameters$par[Parameters$parn == "p2"]
        FittedPsychometricFunctions = Parameters2
    
        FittedPsychometricFunctions$StandardValues = as.character(FittedPsychometricFunctions$StandardValues)
        Psychometric$StandardValues = as.character(Psychometric$StandardValues)
    
    """
    @rget Psychometric FittedPsychometricFunctions
    
    Psychometric[:StandardValuesAsFactor] = "placeholder"
    
    formula1 = @formula(Answer ~ Difference*ConditionOfInterest + (Difference + ConditionOfInterest |ID) + (Difference + ConditionOfInterest|StandardValues));
    GLMM = fit!(GeneralizedLinearMixedModel(formula1, Psychometric, Binomial()), fast=true)
    
    formula2 = @formula(Mean ~ ConditionOfInterest + (1|ID) + (1|StandardValues));
    TwoLevelMean = fit(MixedModel,formula2, FittedPsychometricFunctions)
    
    formula3 = @formula(SD ~ ConditionOfInterest + (1|ID) + (1|StandardValues));
    TwoLevelSD = fit(MixedModel,formula3, FittedPsychometricFunctions)

    [(coeftable(GLMM)).cols[4][2];(coeftable(GLMM)).cols[4][4];(coeftable(TwoLevelMean)).cols[4][2];(coeftable(TwoLevelSD)).cols[4][2]]
    
end

SimulateDataframe_Twolevel (generic function with 1 method)

In [ ]:
ConditionOfInterest = [0;1]
StandardValues = [5;8]
Range_reps = [40,60]
Range_PSE_Difference = [-0.1,-0.05,0,0.05,0.1]
Range_JND_Difference = [-0.2,-0.1,0,0.1,0.2]
Multiplicator_PSE_Standard = 0
Multiplicator_SD_Standard = 0.108
SD_ResponseFunction = 0.1
Mean_Variability_Between = 0.1
SD_Variability_Between = 0.1
nIterations = 100
Range_Participants = [10,12,14,16,18,20]

TotalNumber = length(Range_reps)*length(Range_PSE_Difference)*length(Range_JND_Difference)*length(Range_Participants)
CurrentRunthrough = 0
rightnow = Dates.now()

for reps in Range_reps
    for PSE_Difference in Range_PSE_Difference
        for JND_Difference in Range_JND_Difference
            for n in Range_Participants
                
                TimeStartTrial = Dates.now()
                
                Pvalues_Accuracy = []
                Pvalues_Precision = []
                Pvalues_Accuracy_TwoLevel = []
                Pvalues_Precision_TwoLevel = []
                
                for j in 1:nIterations
                Pvalues = SimulateDataframe_Twolevel(n, 
                                      ConditionOfInterest, 
                                      StandardValues, 
                                      reps, 
                                      PSE_Difference, 
                                      JND_Difference, 
                                      Multiplicator_PSE_Standard, 
                                      Multiplicator_SD_Standard, 
                                      SD_ResponseFunction, 
                                      Mean_Variability_Between, 
                                      SD_Variability_Between)
                    Pvalues_Accuracy = [Pvalues_Accuracy;Pvalues[1]]
                    Pvalues_Precision = [Pvalues_Precision;Pvalues[2]]
                    Pvalues_Accuracy_TwoLevel = [Pvalues_Accuracy_TwoLevel;Pvalues[3]]
                    Pvalues_Precision_TwoLevel = [Pvalues_Precision_TwoLevel;Pvalues[4]]
                end
                
                CurrentRunthrough = CurrentRunthrough + 1

                if CurrentRunthrough == 1

                   global PowerfulDataframe = DataFrame(n=n, 
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],
                        StandardValue2=StandardValues[2], reps=reps, 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, 
                        Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, 
                        power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05),
                        power_Accuracy_Twolevel = mean(Pvalues_Accuracy_TwoLevel .< 0.05),  
                        power_Precision_Twolevel = mean(Pvalues_Precision_TwoLevel .< 0.05),
                        Duration = ((Dates.now()) - TimeStartTrial))   

                else
                    row = DataFrame(n=n, 
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],StandardValue2=StandardValues[2], 
                        reps=reps, 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, 
                        Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, 
                        power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05),
                        power_Accuracy_Twolevel = mean(Pvalues_Accuracy_TwoLevel .< 0.05),  
                        power_Precision_Twolevel = mean(Pvalues_Precision_TwoLevel .< 0.05),
                        Duration=((Dates.now()) - TimeStartTrial))
                    
                    PowerfulDataframe = append!(PowerfulDataframe,row)
                end
                
                print("RUNTHROUGH ", CurrentRunthrough, " out of ", TotalNumber,": ", n, " ", reps, " ", PSE_Difference, " ", JND_Difference, " ", mean(Pvalues_Accuracy .< 0.05), " ", 
                    mean(Pvalues_Precision .< 0.05), " ", PowerfulDataframe[!,:Duration][CurrentRunthrough], " END. ")

            end
            CSV.write(join([reps,"_", PSE_Difference, "_", JND_Difference, ".csv"]),PowerfulDataframe)
        end
    end
end



In [ ]:
function SimulateDataframe(n,
        ConditionOfInterest,
        StandardValues,
        reps,
        PSE_Difference,
        JND_Difference,
        Multiplicator_PSE_Standard,
        Multiplicator_SD_Standard,
        SD_ResponseFunction,
        Mean_Variability_Between,
        SD_Variability_Between)
    
    @rput n ConditionOfInterest StandardValues reps PSE_Difference JND_Difference Multiplicator_PSE_Standard Multiplicator_SD_Standard SD_ResponseFunction Mean_Variability_Between SD_Variability_Between

    TimeBeforeDataframeinR = Dates.now()
    R"""
    ID = paste0("s",1:n)
        Psychometric = SimulatePsychometricFunction_Staircase(ID,
            ConditionOfInterest,
            StandardValues,
            1:reps,
            PSE_Difference,
            JND_Difference,
            Multiplicator_PSE_Standard,
            Multiplicator_SD_Standard,
            SD_ResponseFunction,
            Mean_Variability_Between,
            SD_Variability_Between)
    
        Psychometric$StandardValues = as.character(Psychometric$StandardValues)
    
    """
    @rget Psychometric FittedPsychometricFunctions

    DurationSimulateDataframe = Dates.now() - TimeBeforeDataframeinR
    
    TimeStartTrial = Dates.now()
    
    formula1 = @formula(Answer ~ Difference*ConditionOfInterest + (Difference + ConditionOfInterest |ID) + (Difference + ConditionOfInterest|StandardValues));
    GLMM = fit!(GeneralizedLinearMixedModel(formula1, Psychometric, Binomial()), fast=false)
    
    DurationGLMM = ((Dates.now()) - TimeStartTrial)
    [(coeftable(GLMM)).cols[4][2];(coeftable(GLMM)).cols[4][4];DurationGLMM;DurationSimulateDataframe]
    
end

In [47]:
ConditionOfInterest = [0;1]
StandardValues = [5;8]
Range_reps = [30,40,50,60]
Range_PSE_Difference = 0.1
Range_JND_Difference = 0.2
Multiplicator_PSE_Standard = 0
Multiplicator_SD_Standard = 0.108
SD_ResponseFunction = 0.1
Mean_Variability_Between = 0.1
SD_Variability_Between = 0.1
nIterations = 25
Range_Participants = [10,12,14,16,18,20]

TotalNumber = length(Range_reps)*length(Range_PSE_Difference)*length(Range_JND_Difference)*length(Range_Participants)
CurrentRunthrough = 0
rightnow = Dates.now()

for reps in Range_reps
    for PSE_Difference in Range_PSE_Difference
        for JND_Difference in Range_JND_Difference
            for n in Range_Participants
                
                Pvalues_Accuracy = []
                Pvalues_Precision = []
                DurationsGLMM = []
                DurationsDataframe = []

                for j in 1:nIterations
                Pvalues = SimulateDataframe(n, 
                                      ConditionOfInterest, 
                                      StandardValues, 
                                      reps, 
                                      PSE_Difference, 
                                      JND_Difference, 
                                      Multiplicator_PSE_Standard, 
                                      Multiplicator_SD_Standard, 
                                      SD_ResponseFunction, 
                                      Mean_Variability_Between, 
                                      SD_Variability_Between)
                    Pvalues_Accuracy = [Pvalues_Accuracy;Pvalues[1]]
                    Pvalues_Precision = [Pvalues_Precision;Pvalues[2]]
                    DurationsGLMM = [DurationsGLMM;Dates.value(Pvalues[3])]
                    DurationsDataframe = [DurationsDataframe;Dates.value(Pvalues[4])]
                end
                
                CurrentRunthrough = CurrentRunthrough + 1

                if CurrentRunthrough == 1

                   global PowerfulDataframe = DataFrame(n=n, 
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],
                        StandardValue2=StandardValues[2], reps=reps, 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, 
                        Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, 
                        power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05),
                        Mean_DurationGLMM = mean(DurationsGLMM),
                        SD_DurationGLMM = std(DurationsGLMM),
                        Mean_DurationDataframe = mean(DurationsDataframe),
                        SD_DurationDataframe = std(DurationsDataframe),
                        nIterations = nIterations)

                else
                    row = DataFrame(n=n, 
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],StandardValue2=StandardValues[2], 
                        reps=reps, 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, 
                        Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, 
                        power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05),
                        Mean_DurationGLMM = mean(DurationsGLMM),
                        SD_DurationGLMM = std(DurationsGLMM),
                        Mean_DurationDataframe = mean(DurationsDataframe),
                        SD_DurationDataframe = std(DurationsDataframe),
                        nIterations = nIterations)
                    
                    PowerfulDataframe = append!(PowerfulDataframe,row)
                end

                print("RUNTHROUGH ", CurrentRunthrough, " out of ", TotalNumber,": ", n, " ", reps, " ", 
                    PowerfulDataframe[!,:Mean_DurationGLMM][CurrentRunthrough], " END. ")

            end
        end
    end
end


5RUNTHROUGH 1 out of 24: 10 30 492.08 END. 5RUNTHROUGH 2 out of 24: 12 30 657.44 END. 5RUNTHROUGH 3 out of 24: 14 30 716.48 END. 5RUNTHROUGH 4 out of 24: 16 30 869.4 END. 5RUNTHROUGH 5 out of 24: 18 30 903.76 END. 5RUNTHROUGH 6 out of 24: 20 30 1113.72 END. 5RUNTHROUGH 7 out of 24: 10 40 725.96 END. 5RUNTHROUGH 8 out of 24: 12 40 826.36 END. 5RUNTHROUGH 9 out of 24: 14 40 1047.92 END. 5RUNTHROUGH 10 out of 24: 16 40 1806.08 END. 5RUNTHROUGH 11 out of 24: 18 40 1933.44 END. 5RUNTHROUGH 12 out of 24: 20 40 2041.36 END. 5RUNTHROUGH 13 out of 24: 10 50 887.52 END. 5RUNTHROUGH 14 out of 24: 12 50 1015.4 END. 5RUNTHROUGH 15 out of 24: 14 50 2000.76 END. 5RUNTHROUGH 16 out of 24: 16 50 2244.52 END. 5RUNTHROUGH 17 out of 24: 18 50 2301.52 END. 5RUNTHROUGH 18 out of 24: 20 50 2525.72 END. 5RUNTHROUGH 19 out of 24: 10 60 1292.6 END. 5RUNTHROUGH 20 out of 24: 12 60 1752.84 END. 5RUNTHROUGH 21 out of 24: 14 60 2307.2 END. 5RUNTHROUGH 22 out of 24: 16 60 2597.84 END. 5RUNTHROUGH 23 out of 24: 18 60

"Durations.csv"

In [7]:
GLMM2

Generalized Linear Mixed Model fit by maximum likelihood (nAGQ = 1)
  Answer ~ 1 + Difference + ConditionOfInterest + Difference & ConditionOfInterest + (1 + Difference + ConditionOfInterest | ID) + (1 + Difference + ConditionOfInterest | StandardValues)
  Distribution: Bernoulli{Float64}
  Link: LogitLink()

  Deviance: 7590.1348

Variance components:
                      Column          Variance     Std.Dev.    Corr.
ID             (Intercept)          2.17876557471 1.476064218
               Difference           0.06137904602 0.247747949 -0.14
               ConditionOfInterest  0.02749597991 0.165819118 -0.72 -0.46
StandardValues (Intercept)          0.00016868818 0.012988001
               Difference           0.28217597115 0.531202382 +1.00
               ConditionOfInterest  0.00016319099 0.012774623 -1.00 -1.00

 Number of obs: 8796; levels of grouping factors: 20, 2

Fixed-effects parameters:
───────────────────────────────────────────────────────────────────────
            

In [3]:
ConditionOfInterest = [0;1]
StandardValues = [5;8]
reps = 120
PSE_Difference = 0.1
JND_Difference = 0.2
Multiplicator_PSE_Standard = 0
Multiplicator_SD_Standard = 0.108
SD_ResponseFunction = 0.1
Mean_Variability_Between = 0.1
SD_Variability_Between = 0.1
nIterations = 25
n = 20

20

In [4]:
@rput n ConditionOfInterest StandardValues reps PSE_Difference JND_Difference Multiplicator_PSE_Standard Multiplicator_SD_Standard SD_ResponseFunction Mean_Variability_Between SD_Variability_Between

    R"""
    ID = paste0("s",1:n)
        Psychometric = SimulatePsychometricFunction_Staircase(ID,
            ConditionOfInterest,
            StandardValues,
            1:reps,
            PSE_Difference,
            JND_Difference,
            Multiplicator_PSE_Standard,
            Multiplicator_SD_Standard,
            SD_ResponseFunction,
            Mean_Variability_Between,
            SD_Variability_Between)

    Psychometric$StandardValues = as.character(Psychometric$StandardValues)
    
    """
    @rget Psychometric

,ID,ConditionOfInterest,StandardValues,reps,PSE_Factor_ID,SD_Factor_ID
,Categorical…,Int64,String,Int64,Float64,Float64
1,s2,0,5,1,1.02289,0.763883
2,s3,0,5,1,0.930712,1.09026
3,s4,0,5,1,0.98057,1.13023
4,s5,0,5,1,0.958312,0.913053
5,s7,0,5,1,0.957227,0.924697
6,s8,0,5,1,0.962088,0.933864
7,s9,0,5,1,0.874814,0.969686
8,s10,0,5,1,1.0513,1.0684
9,s11,0,5,1,1.02503,0.984487


In [5]:
 GLMM = fit!(GeneralizedLinearMixedModel(formula1, Psychometric, Bernoulli()))

UndefVarError: UndefVarError: formula1 not defined

In [ ]:
formula1 = @formula(Answer ~ Difference*ConditionOfInterest + 
    (Difference + ConditionOfInterest | ID) + 
    (Difference + ConditionOfInterest | StandardValues));
GLMM2 = fit!(GeneralizedLinearMixedModel(formula1, 
        Psychometric, 
        Bernoulli()),
        verbose = true,
        fast = false)

varyβ = false, obj₀ = 9160.379825759399
   1: 8365.632317236745
   2: 7834.529168897679
   3: 7693.253903724248
   4: 7681.079994645366
   5: 7680.542641084623
   6: 7680.535548289618
   7: 7680.535537275667
   8: 7680.535537275634
f_1: 7680.535537275634 [0.10929039374739623, 1.4041227354936787, -0.7657992350244425, -0.2192165887581911, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0]
varyβ = false, obj₀ = 9178.30578171013
   1: 8531.311687478603
   2: 8132.6105142803
   3: 8829.717311261515
       1, 7792.702340785925
   4: 7700.336437387933
   5: 7681.8248657694285
   6: 7680.570837977189
   7: 7680.549514362241
   8: 7680.549440372774
   9: 7680.54944037132
f_2: 7680.54944037132 [0.21858078749479246, 1.4041227354936787, -0.7657992350244425, -0.2192165887581911, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0]
varyβ = false, obj₀ = 10637.890207020324
   1: Inf
       1, Inf
       2, 11392.471441707441
       3, 9054.763424932778
   2: 8193.495306428704
   3: 77

f_23: 7712.225831977446 [0.10929039374739623, 1.4041227354936787, -0.7657992350244425, -0.2192165887581911, 1.0, -1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0]
varyβ = false, obj₀ = 9159.97675715919
   1: 8406.82734967903
   2: 7901.219916344512
   3: 7746.004805662281
   4: 7715.014627063861
   5: 7713.192763715198
   6: 7713.164412500698
   7: 7713.164303327408
   8: 7713.164303324212
f_24: 7713.164303324212 [0.10929039374739623, 1.4041227354936787, -0.7657992350244425, -0.2192165887581911, 1.0, 0.0, -1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0]
varyβ = false, obj₀ = 9113.204076136033
   1: 8152.792961705707
   2: 7722.2265502582895
   3: 7661.368318503011
   4: 7656.408790397133
   5: 7656.337464916721
   6: 7656.337219749947
   7: 7656.337219742906
f_25: 7656.337219742906 [0.10929039374739623, 1.4041227354936787, -0.7657992350244425, -0.2192165887581911, 1.0, 0.0, 0.0, 0.24999999999999994, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0]
varyβ = false, obj₀ = 9160.376635641454

f_41: 7615.85551929754 [0.10907521652164233, 1.634640919215765, -0.6456179027453968, -0.25741308331522517, 1.3164086561996984, -0.005927852281231605, 0.009791442804162406, 0.3350751321100559, -0.035425944700749966, 0.11027175304591559, 0.49648642454605635, -0.027887726444369428, 0.012706251233824037, 1.2371311365266924, -0.010530001218842755, 1.102731787520117]
varyβ = false, obj₀ = 9221.420547940872
   1: 9412.170686667887
       1, 8058.666532560035
   2: 7670.394764081599
   3: 7616.505522100693
   4: 7613.948969731772
   5: 7613.924950619388
   6: 7613.924906949929
   7: 7613.924906949605
f_42: 7613.924906949605 [0.10915106887001075, 1.8039701700573176, -0.6456883440339624, -0.30719585536909055, 1.3550210260374738, -0.0008271511284562627, 0.03130565613686453, 0.23539927971548938, -0.010054982737671875, 0.0, 0.4693724708722655, -0.014774886091905998, 0.008254623340572637, 1.2133032645131434, 0.031256084910387026, 1.0944303752505082]
varyβ = false, obj₀ = 9147.64680175729
   1: 8965.

   8: 7614.325835145605
   9: 7614.325835145051
f_55: 7614.325835145051 [0.11970071490093062, 1.5873771515342157, -0.606267737916343, -0.2750557151794215, 1.3626176204044465, -0.11931927300044581, -0.023327692882603543, 0.25424397338858634, -0.11197328922311589, 0.0, 0.44388675459536114, -0.03530420425095217, 0.028426661243583066, 1.1526890663649243, -0.06422763985030484, 1.0824123706786073]
varyβ = false, obj₀ = 9127.09479482611
   1: 8582.791316598885
   2: 8265.563478596456
   3: 8974.045241116093
       1, 7717.590121227829
   4: 7622.230654224628
   5: 7614.093264486348
   6: 7613.978731742411
   7: 7613.978308321208
   8: 7613.978308299354
f_56: 7613.978308299354 [0.11974453685609293, 1.5691602908851843, -0.5771498379024027, -0.2687246582036539, 1.3601424657463566, 0.009880027494436696, -0.010517574661372694, 0.22159056707068114, -0.14788890692872964, 0.0, 0.4872266685753899, -0.06524001816698789, 0.019789827265499264, 1.151933201563, -0.03391591430119753, 1.0822479797430804]
var

   6: 7609.390689449421
   7: 7609.39068931529
f_71: 7609.39068931529 [0.12050773220753414, 1.5041599415263485, -0.5742875905263175, -0.29519984890141704, 1.405761700766218, -0.018824295691850963, -0.0486086820075151, 0.25598364224285214, -0.0975141907394493, 0.0, 0.17426917261716096, -0.1171226834755426, 0.0844957199452056, 1.2159644257747475, -0.07970808332186932, 1.1347486233275998]
varyβ = false, obj₀ = 9096.023762637034
   1: 8244.906762424453
   2: 7705.90443855041
   3: 7615.085872669072
   4: 7609.06134671278
   5: 7608.980601063229
   6: 7608.980308960315
   7: 7608.980308947347
f_72: 7608.980308947347 [0.12142548582449503, 1.462972572570173, -0.5739160918188182, -0.3002749160596892, 1.4195417334144054, -0.021916974734993724, -0.037619568483736515, 0.25779415889255397, -0.12660170131554876, 0.03466580847481396, 0.13574279039491408, -0.12334499259175079, 0.11694775413204564, 1.2093810899845594, -0.06130953618939581, 1.1576268431714372]
varyβ = false, obj₀ = 9094.07309895051
   

...

f_261: 7593.279117164554 [0.16295760741300896, 3.463265143184276, -1.369104575433542, -0.4517976858086432, 1.4343668724679044, -0.03375341420358372, -0.11649944598107713, 0.24217973412219979, -0.07742704351793875, 0.00010209950774783239, 0.0, -0.06457889216058119, -0.005957294554851009, 1.48325390563937, -0.009285922718215446, 0.0024992443102866855]
varyβ = false, obj₀ = 11182.729180130305
   1: Inf
       1, 13499.200149332517
       2, 8483.951827489973
   2: 7655.465039684285
   3: 7595.455522495661
   4: 7593.2678983496835
   5: 7593.25092499637
   6: 7593.250899327986
   7: 7593.250899327857
f_262: 7593.250899327857 [0.1633240471556761, 3.461009824465174, -1.3670709434651893, -0.4513098349139432, 1.4332936495439517, -0.035467202531419106, -0.11791390079040907, 0.2440287311113802, -0.07813075650457454, 0.0, 0.0, -0.0646890150747703, -0.001348054254517557, 1.4774526216605948, -0.0029295208864418706, 0.0]
varyβ = false, obj₀ = 11192.669095521811
   1: Inf
       1, 13549.43786592804


f_277: 7593.180337907871 [0.16448241160190402, 3.434068198048516, -1.366298898435937, -0.4510504084298059, 1.4355816892423887, -0.038818185063494, -0.11790317112888551, 0.24616465920706032, -0.0913473980081208, 0.012320156934635302, 0.0, -0.06748074393938938, 0.0006146902279954926, 1.4630622093975658, 0.000177389914813841, 0.001570251896918003]
varyβ = false, obj₀ = 11134.965561580915
   1: Inf
       1, 13485.94898379912
       2, 8513.297315326505
   2: 7664.705157107123
   3: 7596.167797395561
   4: 7593.218291892341
   5: 7593.192463567544
   6: 7593.192410734983
   7: 7593.192410734423
f_278: 7593.192410734423 [0.1650395511620714, 3.434516293036058, -1.3653099942463747, -0.4516124720660711, 1.4354050298291623, -0.03779313900870084, -0.11395309202396331, 0.2510175230599575, -0.09104392210785567, 0.01347327147431604, 0.0, -0.06930011468127827, -0.0004441321083997517, 1.4632263969625068, 0.0003873158556584986, 0.0018229791254678347]
varyβ = false, obj₀ = 11134.076786249743
   1: Inf


   3: 7595.547806464346
   4: 7593.164962513267
   5: 7593.145035831936
   6: 7593.144999705209
   7: 7593.144999704933
f_291: 7593.144999704933 [0.16693637610491915, 3.3605876575092886, -1.3506824387615715, -0.4500594638133763, 1.4419944022322229, -0.03086424208014492, -0.11153143191960026, 0.2490292700438633, -0.10746884991931142, 0.021756019130994446, 0.0, -0.06067163677689005, -0.0018847232343814908, 1.4389046884394527, 0.017881135259217674, 0.0]
varyβ = false, obj₀ = 10998.980954885727
   1: Inf
       1, 12832.844092637088
       2, 8446.906496675172
   2: 7653.957412637938
   3: 7595.313754744652
   4: 7593.102438991385
   5: 7593.084270330753
   6: 7593.084239319938
   7: 7593.0842393197345
f_292: 7593.0842393197345 [0.1665432079494391, 3.3550074662915828, -1.3468515083454229, -0.4496449061779228, 1.446173054399955, -0.033635641353639216, -0.11849120071887113, 0.24876211858024933, -0.09608329879978457, 0.02173063967501735, 0.0, -0.06365717903583186, -0.005812593802238634, 1.436

f_305: 7593.145498247812 [0.1782993852634158, 3.1771734660613093, -1.3473326784970479, -0.4518154782736885, 1.4677451104880403, -0.03850225437523572, -0.15958116332581357, 0.24168711825876157, -0.08793754474910478, 0.07882898847290694, 0.0027913875053523977, -0.05208149196768068, -0.017265885480751423, 1.3695098030999926, 0.016635748812106162, 0.005765795283900427]
varyβ = false, obj₀ = 10622.728139453833
   1: 27422.11579342375
       1, 11282.756266949791
       2, 8315.606291146707
   2: 7650.370495535202
   3: 7594.687241381368
   4: 7592.811039770413
   5: 7592.796017057996
   6: 7592.795994064528
   7: 7592.7959940644105
f_306: 7592.7959940644105 [0.18027464164366278, 3.138950015685319, -1.3432176108493339, -0.45559167920983196, 1.4833660865623712, -0.023768266365198628, -0.1410172444347649, 0.24502673380333548, -0.08594875648083491, 0.07724653295893219, 0.0, -0.04498209605020977, -0.009494145238254766, 1.3763550024204163, 0.0303597716486835, 0.0]
varyβ = false, obj₀ = 10614.9155

   1: 27114.635753554594
       1, 11308.25989820913
       2, 8323.962730580626
   2: 7652.1520429573675
   3: 7594.786018952155
   4: 7592.67644844047
   5: 7592.6582451078
   6: 7592.658212414326
   7: 7592.65821241409
f_319: 7592.65821241409 [0.1800574902631819, 3.113849998235949, -1.3601263470701899, -0.4531625121940559, 1.4852959744240757, -0.025839630675095804, -0.13446689695267094, 0.25213629566276935, -0.09203945458545404, 0.08485198782462051, 0.012292399374832782, -0.04895683274327206, -0.00748779215278804, 1.3740833062524251, 0.0002513543367948238, 0.0]
varyβ = false, obj₀ = 10584.654928173059
   1: 27008.327200305535
       1, 11231.517143265597
       2, 8312.154514113568
   2: 7651.948502163006
   3: 7594.753961249644
   4: 7592.678619944026
   5: 7592.6609588535875
   6: 7592.660928208961
   7: 7592.660928208756
f_320: 7592.660928208756 [0.17965425431809343, 3.117079801491112, -1.3604584231122994, -0.45371753927172237, 1.4865230562405416, -0.027724666353162716, -0.131864

In [53]:
verbaggform = @formula(r2 ~ (m|id) + (m|item));
gm1a = fit!(GeneralizedLinearMixedModel(verbaggform, 
        dat[:VerbAgg],
        Bernoulli()), 
        verbose = true,
        fast = false)

varyβ = true, obj₀ = 11537.4321647439, β = [-0.09553673453397096]
   1: 8424.125470328892
   2: 8175.430301981829
   3: 8156.265904211067
   4: 8155.651353611125
   5: 8155.648278214871
   6: 8155.648277948747
f_1: 8155.648277948747 [1.0, 0.0, 1.0, 1.0, 0.0, 1.0]
varyβ = true, obj₀ = 11844.83844993634, β = [-0.06089628432231471]
   1: 8514.668180878241
   2: 8176.001253069019
   3: 8138.994145532149
   4: 8136.989074175192
   5: 8136.962203379701
   6: 8136.962188565084
   7: 8136.962188565075
f_2: 8136.962188565075 [1.75, 0.0, 1.0, 1.0, 0.0, 1.0]
varyβ = true, obj₀ = 11664.00273650668, β = [-0.052070779955305624]
   1: 8603.072090222
   2: 8264.207958870173
   3: 8219.01434959314
   4: 8216.246613253423
   5: 8216.206416563964
   6: 8216.206391322778
   7: 8216.206391322748
f_3: 8216.206391322748 [1.0, 1.0, 1.0, 1.0, 0.0, 1.0]
varyβ = true, obj₀ = 11801.03848743459, β = [-0.07839938514016795]
   1: 8563.993202132338
   2: 8243.739833434034
   3: 8204.823112447244
   4: 8202.5063753555

   5: 8118.206834037523
   6: 8118.206833359988
f_27: 8118.206833359988 [1.4263828375877787, -0.14999374003097815, 1.0126564745353694, 1.1062777033694022, -0.41647333647660273, 1.0778863328881196]
varyβ = true, obj₀ = 11737.59382194211, β = [-0.11939078464599105]
   1: 8450.440298879446
   2: 8147.092848393851
   3: 8119.250068634069
   4: 8118.137986440358
   5: 8118.130705911973
   6: 8118.1307050709265
f_28: 8118.1307050709265 [1.4438251002810139, -0.15348064568604142, 1.0385986195331278, 1.0992238268786267, -0.3276158711274158, 1.0901286203841567]
varyβ = true, obj₀ = 11752.97593718699, β = [-0.10335056719202707]
   1: 8457.472830164787
   2: 8148.876230882531
   3: 8120.023305752707
   4: 8118.848087651869
   5: 8118.840233040019
   6: 8118.840232088568
f_29: 8118.840232088568 [1.4269147688416433, -0.1505579175761898, 1.0928013967701544, 1.1278277746357874, -0.28912444786418623, 1.062839161785352]
varyβ = true, obj₀ = 11738.507511783111, β = [-0.10905352474863772]
   1: 8452.08343

   2: 8146.748503233587
   3: 8118.910362683137
   4: 8117.794474441589
   5: 8117.787025931521
   6: 8117.787025033422
f_51: 8117.787025033422 [1.4326421617574732, -0.12205362112039941, 1.0146899672029583, 1.0273981248633186, -0.32157642502655426, 1.1844124489782673]
varyβ = true, obj₀ = 11725.43050816685, β = [-0.07051898261309597]
   1: 8450.368712624482
   2: 8146.815297210567
   3: 8118.920939935166
   4: 8117.798725184261
   5: 8117.791153127116
   6: 8117.791152195739
f_52: 8117.791152195739 [1.436524837259671, -0.11753259705008591, 1.0079861488769923, 1.0111357755607255, -0.30208754530870974, 1.203130771408462]
varyβ = true, obj₀ = 11723.038403628443, β = [-0.061234240952403055]
   1: 8448.839176507134
   2: 8146.445640849248
   3: 8118.876838062272
   4: 8117.7828719011495
   5: 8117.77571625899
   6: 8117.7757154224255
f_53: 8117.7757154224255 [1.4191420495266462, -0.12500918822560486, 1.0174725462830674, 1.0256310505846835, -0.32630710603910557, 1.188204498826188]
varyβ = tr

f_75: 8117.707193062496 [1.4225795978197728, -0.12079430523622134, 1.0193361419667295, 0.9691502973312868, -0.2846411414347507, 1.2347413494905704]
varyβ = true, obj₀ = 11727.907532067218, β = [-0.0449910002483549]
   1: 8450.31616960362
   2: 8146.664887823077
   3: 8118.829018567289
   4: 8117.714653858872
   5: 8117.707222230049
   6: 8117.7072213283445
f_76: 8117.7072213283445 [1.4231932891256351, -0.12113065029667047, 1.0197280130009534, 0.969905900726263, -0.2838811941500491, 1.234720348004867]
varyβ = true, obj₀ = 11727.493881699218, β = [-0.04507185654415512]
   1: 8450.12345275044
   2: 8146.625354588958
   3: 8118.825569214912
   4: 8117.713816363279
   5: 8117.706415484986
   6: 8117.706414589882
f_77: 8117.706414589882 [1.4231072455223108, -0.12217936983411196, 1.0190629316065032, 0.9693446801984186, -0.28373919229766564, 1.2343320427476048]
varyβ = true, obj₀ = 11727.394367762738, β = [-0.04498545252878663]
   1: 8450.011738079205
   2: 8146.5983222634795
   3: 8118.823049

   6: 8117.703847214434
f_98: 8117.703847214434 [1.4248780094856377, -0.12509068799669643, 1.0180026882621243, 0.9693739579319673, -0.3019611063198081, 1.2291756437805155]
varyβ = true, obj₀ = 11726.804061897054, β = [-0.04779418570859973]
   1: 8449.843260008662
   2: 8146.563978754355
   3: 8118.818850974308
   4: 8117.711203433346
   5: 8117.703858460553
   6: 8117.703857580467
f_99: 8117.703857580467 [1.4248747129403116, -0.12492384209757453, 1.0180119075447072, 0.969203669352264, -0.30231798453135617, 1.2290179054382884]
varyβ = true, obj₀ = 11726.779615907779, β = [-0.04782774504469326]
   1: 8449.823912095342
   2: 8146.559471701655
   3: 8118.818499673229
   4: 8117.71118231446
   5: 8117.7038414808885
   6: 8117.703840601615
f_100: 8117.703840601615 [1.4248041593109801, -0.1251616601424228, 1.0180426864076375, 0.9695013594057501, -0.30227596310053173, 1.2293373575977198]
varyβ = true, obj₀ = 11726.77859426012, β = [-0.047823842939792284]
   1: 8449.82152465385
   2: 8146.55886

f_121: 8117.703824111219 [1.4248290795943621, -0.1252024768776387, 1.018402178050455, 0.9696047770194933, -0.30237210237058265, 1.2293380422275606]
varyβ = true, obj₀ = 11726.92583127275, β = [-0.047862425652792466]
   1: 8449.871835332218
   2: 8146.567805327317
   3: 8118.818993138106
   4: 8117.711170721324
   5: 8117.70382497372
   6: 8117.703824093508
f_122: 8117.703824093508 [1.4248330564999412, -0.1252051911539735, 1.018399078339009, 0.9695796042632286, -0.30233080895093717, 1.2293615443994297]
varyβ = true, obj₀ = 11726.929586894943, β = [-0.047848098800483706]
   1: 8449.872577880646
   2: 8146.567912225674
   3: 8118.818999921893
   4: 8117.711170805775
   5: 8117.703824975262
   6: 8117.703824095032
f_123: 8117.703824095032 [1.4248352132881406, -0.12520594774736168, 1.0183996928724515, 0.9695727605777168, -0.3023362042492962, 1.229376097815144]
varyβ = true, obj₀ = 11726.925207831357, β = [-0.04784438659914994]
   1: 8449.871177162948
   2: 8146.5676727826485
   3: 8118.8189

   4: 8117.711207691849
   5: 8117.703848052738
   6: 8117.703847170782
f_20: 8117.703847170782 [-0.05149609645305187, 1.4257835698587167, -0.12506466141495268, 1.018939286415677, 0.9666526357285878, -0.30107148963877606, 1.2301970573183008]
varyβ = false, obj₀ = 11728.111558663612
   1: 8450.448458257824
   2: 8146.677993576473
   3: 8118.829151067292
   4: 8117.714216518982
   5: 8117.706789019978
   6: 8117.706788124644
f_21: 8117.706788124644 [-0.0513827350267183, 1.428977758181214, -0.1250598815859572, 1.0189723382061215, 0.9706578662938341, -0.3070509805218045, 1.2272768003165828]
varyβ = false, obj₀ = 11726.375326240839
   1: 8449.84733356655
   2: 8146.568695460532
   3: 8118.818718536116
   4: 8117.7109254426905
   5: 8117.703582208337
   6: 8117.703581328972
f_22: 8117.703581328972 [-0.05200147726309854, 1.4247862583958264, -0.12507652863007654, 1.0188065599525542, 0.9696783462672147, -0.2952942644030685, 1.2295527039937653]
varyβ = false, obj₀ = 11726.390359458503
   1: 8449

Generalized Linear Mixed Model fit by maximum likelihood (nAGQ = 1)
  r2 ~ 1 + (1 + m | id) + (1 + m | item)
  Distribution: Bernoulli{Float64}
  Link: LogitLink()

  Deviance: 8117.7034

Variance components:
        Column    Variance   Std.Dev.    Corr.
id   (Intercept)  1.76802958 1.32967274
     m: do        0.91685601 0.95752598 -0.12
item (Intercept)  0.81884174 0.90489875
     m: do        1.39470367 1.18097573 -0.24

 Number of obs: 7584; levels of grouping factors: 316, 24

Fixed-effects parameters:
──────────────────────────────────────────────────────
               Estimate  Std.Error    z value  P(>|z|)
──────────────────────────────────────────────────────
(Intercept)  -0.0528834     0.2291  -0.230832   0.8174
──────────────────────────────────────────────────────

In [62]:
Pkg.add("MixedModels#master")

Pkg.Types.PkgError: `MixedModels#master` is not a valid package name

In [34]:
verbaggform = @formula(r2 ~ (m|id)+(m|item));
gm1a = fit!(GeneralizedLinearMixedModel(verbaggform, dat[:VerbAgg],
    Bernoulli()), fast=false)

Generalized Linear Mixed Model fit by maximum likelihood (nAGQ = 1)
  r2 ~ 1 + a + g + b + s + (1 + m | id) + (1 + m | item)
  Distribution: Bernoulli{Float64}
  Link: LogitLink()

  Deviance: 8064.2288

Variance components:
        Column     Variance   Std.Dev.   Corr.
id   (Intercept)  1.801874970 1.3423394
     m: do        0.896334055 0.9467492 -0.23
item (Intercept)  0.041683269 0.2041648
     m: do        0.639784817 0.7998655 -0.09

 Number of obs: 7584; levels of grouping factors: 316, 24

Fixed-effects parameters:
──────────────────────────────────────────────────────
               Estimate  Std.Error    z value  P(>|z|)
──────────────────────────────────────────────────────
(Intercept)   0.459534   0.363961     1.26259   0.2067
a             0.0531583  0.0163353    3.2542    0.0011
g: M          0.243634   0.186424     1.30688   0.1913
b: scold     -1.05134    0.163385    -6.43477   <1e-9
b: shout     -1.76674    0.164763   -10.7229    <1e-26
s: self      -1.028      0.1335

In [41]:
using RData
datf = joinpath(dirname(pathof(MixedModels)),"..","test","dat.rda")
const dat = Dict(Symbol(k)=>v for (k,v) in load(datf))

7584-element CategoricalArray{String,1,UInt8}:
 "N"
 "N"
 "Y"
 "Y"
 "Y"
 "Y"
 "Y"
 "N"
 "N"
 "Y"
 "Y"
 "Y"
 "Y"
 ⋮
 "N"
 "Y"
 "N"
 "N"
 "N"
 "N"
 "N"
 "N"
 "N"
 "N"
 "N"
 "N"